## The problem with small cap indices
The rise of passive investing in the last decades has sparked much debate about whether stocks are still priced appropriately in the market (see e.g. [this video](https://www.youtube.com/watch?v=Wv0pJh8mFk0) for a discussion). As a matter of fact, the aggregate PE ratio of large cap US stocks in the S&P 500 has been [hovering around 20-25](https://www.longtermtrends.net/price-earnings-ratio/) for the past 5 years now, a level which some consider indicative of a bubble — although it is unclear whether this is due to passive investing or other factors such as equity buybacks fuelled by low interest rates, ageing demographics, etc. But what about small cap stocks? Small cap stocks have received much attention recently as they have outperformed large cap stocks in the US over the past [15 years](https://www.longtermtrends.net/large-cap-vs-small-cap/). US small cap stocks may represent a good investment opportunity, but some also argue that they are currently highly overvalued (e.g. [this](https://www.youtube.com/watch?v=_pqz_dj7zl0), [this](https://www.thestreet.com/investing/stocks/will-the-real-pe-ratio-please-stand-up-14923577) or [this](https://seekingalpha.com/article/4111021-russell-2000-bubble)). 

Index funds tracking small caps are becoming more popular with the rise of factor investing. However, it is not always easy to get a clear picture of the quality of the firms included in the underlying index. The issue with aggregate PE ratios for small cap stocks indices is that a significant share of small cap firms might have negative earnings, and those are usually excluded from aggregate PE ratio computations. For example, the aggregate PE ratio (excluding negative earnings) for the Russell 2000 index is 15.71 in April 2020 according to the [factsheet](https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssue?issueName=US2000USD) provided by FTSE Russell. Not so bad ... but according to [bloomberg](https://www.bloomberg.com/quote/RTY:IND), the PE ratio is actually closer to 45, and the WSJ also publishes a [similar figure](https://www.wsj.com/market-data/stocks/peyields). So, are US small cap stocks overvalued? How many firms in the Russell 2000 actually have negative earnings? What is the aggregate PE ratio of the Russell 2000 index if these firms are included in the computation? This post attempts to answer these questions by gathering information on company fundamentals for all firms in the Russell 2000 index. The methods shown here can easily be adapted to other index providers such as MSCI or STOXX.

## Obtaining index constituents

Index providers are required by ESMA guidelines to disclose a list of constituents of their indices: this means that it is usually possible to get a list of firms included in an index. The list of firms included in the Russell 2000 is available [here](https://research.ftserussell.com/analytics/factsheets/Home/DownloadConstituentsWeights/?indexdetails=US2000) in pdf format. We can use the python package `tabula` to read the pdf and extract the list of constituents directly as a dataframe.

In [257]:
import requests as rq
import tabula as tb
import tempfile as tf
import pandas as pd

url = "https://research.ftserussell.com/analytics/factsheets/Home/DownloadConstituentsWeights/?indexdetails=US2000"
with tf.NamedTemporaryFile() as f:
    f.write(rq.get(url).content)
    f.flush()
    t = tb.read_pdf(f.name, pages='all', silent=True)

constituents = pd.concat(t).reset_index(drop=True)
constituents.columns = ['name', 'weight', 'country']
display(constituents)

,name,weight,country
0,1-800 Flowers Com,0.017,United States
1,1st Constitution Bancorp,0.009,United States
2,1st Source Corp,0.035,United States
3,22nd Century Group Inc,0.006,United States
4,3d Systems Corp,0.048,United States
...,...,...,...
1990,Zogenix Inc,0.106,United States
1991,Zumiez Inc,0.033,United States
1992,Zuora,0.060,United States
1993,Zynerba Pharmaceuticals,0.007,United States


## Getting ticker codes

This provides us with the names of all 1995 companies in the Russell 2000 index, as well as with their weight in the index. However at this stage we lack a standard identifier for the firms, which makes it hard to get financial information. A convenient way to get a firm's ticker or [RIC](https://en.wikipedia.org/wiki/Reuters_Instrument_Code) is to use the OpenPermID `search` [API](https://developers.refinitiv.com/open-permid/permid-entity-search). OpenPermID is a database of entities (organizations, persons, financials instruments, etc.) curated by Refinitiv. It comes with a python package, making it very easy to use. You will need to ask for a free API token, which allows for up to 5,000 requests per day (enough in our case). The `search` API will return matches in three categories (organizations, instruments, and quotes), with up to 5 matches in each category (by default) for a given query. The only issue is that it is quite slow, as the API limits queries to 1 or 2 per second for a free token.

In [258]:
from OpenPermID import OpenPermID
from collections import defaultdict
from time import sleep
import logging
from tqdm import tqdm_notebook
from IPython.display import Markdown

# Initialize the OpenPermID api
TOKEN = '***REMOVED***'
opid = OpenPermID()
opid.set_access_token(TOKEN)
opid.set_log_level(logging.CRITICAL)

# Search the OpenPermID database
entities = defaultdict(list)
for r in tqdm_notebook(constituents.itertuples(), total=constituents.shape[0]):
    data, error = opid.search(r.name)
    if (error is not None):
        # Maybe we exceeded the rate limit: wait and retry
        sleep(5)
        data, error = opid.search(r.name)
    if type(data)==dict:
        # the API returns a dict with relevant matches for each category
        for k, v in data.items():
            v['id'] = r.Index
            entities[k].append(v)

# Print the first match in each category
entities = {k: pd.concat(v) for k,v in entities.items()}
for k,v in entities.items():
    display(Markdown(f'* **Matches for {k}:**'))
    display(v.loc[0].head())

* **Matches for organizations:**

,@id,organizationName,primaryTicker,orgSubtype,hasHoldingClassification,hasURL,id
0,https://permid.org/1-4295912343,1-800-Flowers.Com Inc,FLWS,Company,publiclyHeld,http://ww11.1800flowers.com/,0
0,https://permid.org/1-4295901920,1st Constitution Bancorp,FCCY,Bank/Financial Holding Company,publiclyHeld,http://www.1stconstitution.com/investorRelations/,1
0,https://permid.org/1-4295912337,1st Source Corp,SRCE,Bank/Financial Holding Company,publiclyHeld,https://www.1stsource.com,2
0,https://permid.org/1-4296233069,22nd Century Group Inc,XXII,Company,publiclyHeld,http://www.xxiicentury.com/,3
0,https://permid.org/1-4295912347,3D Systems Corp,DDD,Company,publiclyHeld,https://www.3dsystems.com/,4


* **Matches for instruments:**

,@id,hasName,assetClass,isIssuedByName,isIssuedBy,hasPrimaryQuote,primaryTicker,id
0,https://permid.org/1-8590930876,1-800 FLOWERS.COM Ord Shs Class A,Ordinary Shares,1-800-Flowers.Com Inc,https://permid.org/1-4295912343,https://permid.org/1-55835327192,FLWS,0
0,https://permid.org/1-8589939703,1st Constitution Bancorp Ord Shs,Ordinary Shares,1st Constitution Bancorp,https://permid.org/1-4295901920,https://permid.org/1-55835325961,FCCY,1
0,https://permid.org/1-8589959590,1st Source Ord Shs,Ordinary Shares,1st Source Corp,https://permid.org/1-4295912337,NaN,NaN,2
0,https://permid.org/1-8590615928,Century Group 22 Ord Shs,Ordinary Shares,22nd Century Group Inc,https://permid.org/1-4296233069,NaN,NaN,3
0,https://permid.org/1-21716781318,89bio Ord Shs,Ordinary Shares,89Bio Inc,https://permid.org/1-5071508069,https://permid.org/1-25727362195,ETNB,5


* **Matches for quotes:**

,@id,hasName,assetClass,isQuoteOfInstrumentName,hasRIC,hasMic,hasExchangeTicker,isQuoteOf,id
0,https://permid.org/1-55835327192,1-800 FLOWERS.COM CL A ORD,Ordinary Shares,1-800 FLOWERS.COM Ord Shs Class A,FLWS.OQ,XNGS,FLWS,https://permid.org/1-8590930876,0
0,https://permid.org/1-55835325961,1ST CONSTITUTION BANCORP ORD,Ordinary Shares,1st Constitution Bancorp Ord Shs,FCCY.OQ,XNMS,FCCY,https://permid.org/1-8589939703,1
0,https://permid.org/1-55835355439,1ST SOURCE ORD,Ordinary Shares,1st Source Ord Shs,SRCE.OQ,XNGS,SRCE,https://permid.org/1-8589959590,2
0,https://permid.org/1-21538363135,22nd CENTURY GROUP ORD,Ordinary Shares,Century Group 22 Ord Shs,XXII.A,XASE,XXII,https://permid.org/1-8590615928,3
0,https://permid.org/1-25727362195,89BIO ORD,Ordinary Shares,89bio Ord Shs,ETNB.OQ,XNMS,ETNB,https://permid.org/1-21716781318,5


I will favor the organization ticker of the first match when it is available, then the instrument and finally the quote ticker if any. We can also gather the corresponding names returned by OpenPermID to gauge the quality of the match — this could still be done by exhaustive inspection of the data since there are only a couple thousand constituents.

In [414]:
# Format the information on names and tickers
ent = dict()
ent['org'] = (entities['organizations']
              .rename({'organizationName': 'opid_name',
                       'primaryTicker': 'opid_ticker'}, axis=1))
ent['ins'] = (entities['instruments']
            .rename({'primaryTicker': 'opid_ticker'}, axis=1)
            .assign(opid_name = entities['instruments'].isIssuedByName
                    .fillna(entities['instruments'].hasName)))
ent['quo'] = (entities['quotes']
            .rename({'hasExchangeTicker': 'opid_ticker'}, axis=1)
            .assign(opid_name = entities['quotes'].isQuoteOfInstrumentName
                    .fillna(entities['quotes'].hasName)))

# Pick the ticker
for c in ['org', 'ins', 'quo']:
    d = (ent[c]
         .groupby('id')
         .first()
         .loc[:,['opid_ticker', 'opid_name']]
        )
    if c=='org':
        info = d
    else:
        info.update(d, overwrite=False)

# Add to the data and display for inspection
constituents = pd.concat([info, constituents], axis=1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(constituents.sort_values('weight', ascending=False))

,opid_ticker,opid_name,name,weight,country
1307,NVCR,Novocure Ltd,Novocure Ltd,0.348,United States
1165,MDCO,Medicines Co,Medicines Co,0.305,United States
779,GNRC,Generac Holdings Inc,Generac Holdings Inc,0.292,United States
1110,LITE,Lumentum Holdings Inc,Lumentum Holdings Inc,0.291,United States
1764,TDOC,Teladoc Health Inc,Teladoc Health Inc,0.287,United States
166,ARWR,Arrowhead Pharmaceuticals Inc,Arrowhead Pharmaceutical,0.281,United States
851,HAE,Haemonetics Corp,Haemonetics Corp,0.280,United States
1143,NaN,NaN,Marriott Vac World Corp,0.253,United States
1414,PFGC,Performance Food Group Co,Performance Food Group,0.252,United States
1814,TREX,Trex Company Inc,Trex Inc,0.251,United States


### Assessing match quality

To help with data inspection, we can compute a string distance between the original company name and the one matched from OpenPermID. The Levenshtein distance is usually a popular option when it comes to string distances, but it is not particularly well suited in this case since we can easily have additional words in one name or the other. A more solid choice might be to use a set-based distance such as the overlap between the two names' bigrams, defined as the number of common bigrams divided by the number of bigrams in the shorter of the two names (hence simple word additions will not count). As it turns out, there seems to be quite a few mismatches at first glance, and things look a bit complex. Among high-weight constituents, Brinks co gets matched to JPMorgan Chase Bank NA, but the ticker BCO is actually the correct one. Another example is Science Applications Int: this company was split in two in 2013, with one retaining the name, and the other renamed as Leidos, hence the match that we get — although there is no way to know whether this is actually correct. Overall, matches with doubtful quality only represent about 3% of the index, and a lot of these are probably correct as we saw from the previous examples.

In [415]:
import string

def clean_string(s):
    """Remove punctuation and make lowercase."""
    
    if type(s)==str:
        table = str.maketrans(dict.fromkeys(string.punctuation)) 
        return s.translate(table).lower()
    return s

def overlap(x, y):
    """Compute the overlap between bigrams of two strings."""
    
    try:
        s1 = set(ngrams(clean_string(x), 2))
        s2 = set(ngrams(clean_string(y), 2))
        return len(s1.intersection(s2)) / min(len(s1), len(s2))
    except:
        return np.nan

constituents['name_dist'] = constituents.apply(lambda x: overlap(x.loc['opid_name'], x.loc['name']), axis=1)
uncertain = constituents[constituents.name_dist.lt(.7) & constituents.name_dist.notna()]
print(f'Uncertain matches represent {uncertain.weight.sum():.2f}% of the index')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(uncertain.sort_values('name_dist'))

Uncertain matches represent 3.26% of the index


,opid_ticker,opid_name,name,weight,country,name_dist
353,MOTO,Classified Ventures LLC,Cars.com,0.039,United States,0.000000
521,CMLS,CM Wind Down Topco LLC,Cumulus Media Inc,0.012,United States,0.062500
1467,PRGS,Peerdirect Inc,Progress Software Corp,0.088,United States,0.076923
952,IBRC,iBrands Corp,Innovative Industrial,0.041,United States,0.090909
944,006490,Inscobee Inc,Industrial Logistics,0.069,United States,0.100000
1783,STO,Santos Ltd,The Joint Corp,0.010,United States,0.111111
946,GE,General Electric Co,Information Services Grp,0.004,United States,0.111111
1610,SGMS,Autotote Interactive Inc,Scientific Games Corp,0.071,United States,0.111111
140,APX,Apex Resources Inc,Appian,0.062,United States,0.200000
399,EAT,Brinker International Inc,Cheesecake Factory Inc,0.077,United States,0.210526


Another thing we can look at to assess match quality is whether we have any duplicated tickers. As it turns out, this happens in a number of cases, and most of these seem to indicate matching errors. I will correct these cases by hand since there are only a few.

In [416]:
from io import StringIO

# Find duplicated tickers
duplicated = (constituents[constituents.opid_ticker.notna() 
                           & constituents.opid_ticker.duplicated(keep=False)]
              .sort_values('opid_ticker'))
display(duplicated)

# Correct by hand
dups = StringIO(
"""
name,ticker
National Bankshares Inc, NKSH
Dsp Group Inc, DSPG
Cheesecake Factory Inc, CAKE
First Bancorp Inc, FNLC
Hertz Global Holdings, HTZ
Box Inc, BOX
National Gen Hldgs Corp, NGHC
Century Bancorp Inc, CNBKA
Bancorp Inc, TBBK
Tetra Technologies Inc, TTI
"""
)
dups = pd.read_csv(dups)
constituents = constituents.merge(dups, how='left', on='name')
constituents = constituents.assign(ticker=constituents.ticker.fillna(constituents.opid_ticker))

,opid_ticker,opid_name,name,weight,country,name_dist
105,AMNB,American National Bankshares Inc,American Natl Bankshares,0.020,United States,0.954545
1243,AMNB,American National Bankshares Inc,National Bankshares Inc,0.013,United States,1.000000
380,CENT,Central Garden & Pet Co,Central Garden & Pet Co,0.059,United States,1.000000
381,CENT,Central Garden & Pet Co,Central Garden & Pet Co,0.015,United States,1.000000
391,CEVA,CEVA Inc,Ceva Inc,0.028,United States,1.000000
577,CEVA,CEVA Inc,Dsp Group Inc,0.016,United States,0.428571
299,EAT,Brinker International Inc,Brinker International,0.075,United States,1.000000
399,EAT,Brinker International Inc,Cheesecake Factory Inc,0.077,United States,0.210526
694,FBNC,First Bancorp (North Carolina),First Bancorp Nc,0.056,United States,1.000000
693,FBNC,First Bancorp (North Carolina),First Bancorp Inc,0.015,United States,0.933333


### Tickers for firms with no match

Now let's look at firms for which OpenPermID did not find a ticker. They represent about 7% of the total index, but about half of that is concentrated in the top 20 firms. We can significantly improve the data with minimal work by looking these up manually.

In [417]:
unmatched = constituents[constituents.opid_ticker.isna()].sort_values('weight', ascending=False)
print(f'Unmatched firms account for {unmatched.weight.sum():.2f}% of the index')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(unmatched.assign(cum_weight = unmatched.weight.cumsum()))

Unmatched firms account for 7.31% of the index


,opid_ticker,opid_name,name,weight,country,name_dist,ticker,cum_weight
1143,NaN,NaN,Marriott Vac World Corp,0.253,United States,NaN,NaN,0.253
711,NaN,LIGHT IND PPTY(FIRST IND RLTY),First Industrial Rlty Tr,0.249,United States,0.571429,NaN,0.502
1550,NaN,NaN,Rexford Industrial Rlty,0.240,United States,NaN,NaN,0.742
268,NaN,NaN,Blackstone Mortgage Trst,0.223,United States,NaN,NaN,0.965
798,NaN,NaN,Global Blood Thera,0.219,United States,NaN,NaN,1.184
1522,NaN,NaN,Reate Pharmaceuticals,0.217,United States,NaN,NaN,1.401
707,NaN,NaN,First Finl Bankshares,0.217,United States,NaN,NaN,1.618
1702,NaN,STAG Industrial Operating Partnership LP,Stag Indl Inc,0.201,United States,0.800000,NaN,1.819
1854,NaN,NaN,Unifirst Corp Mass,0.145,United States,NaN,NaN,1.964
453,NaN,"COLUMBIA BKG FED-BATH,PAINTED",Columbia Bkg System Inc,0.139,United States,0.636364,NaN,2.103


In [418]:
addition = StringIO(
"""
name,ticker
Marriott Vac World Corp,VAC
First Industrial Rlty Tr,FR
Rexford Industrial Rlty,REXR
Blackstone Mortgage Trst,BXMT
Global Blood Thera,GBT
Reate Pharmaceuticals,RETA
First Finl Bankshares,FFIN
Stag Indl Inc,STAG
Unifirst Corp Mass,UNF
Columbia Bkg System Inc,COLB
Washington Fed Inc,WAFD
Apollo Commercial Re,ARI
Piedmont Office Rlty Tr,PDM
Spirit Airls Inc,SAVE
Independent Bank Ma,INDB
Summit Matls Inc,SUM
American Equity Invst,AEL
"""
)
addition = pd.read_csv(addition)
constituents = constituents.merge(addition, how='left', on='name', suffixes=('','_new'))
constituents = constituents.assign(ticker=constituents.ticker.fillna(constituents.ticker_new))

## Getting financial information on companies

Now that we have gathered tickers for almost all firms in the index, we can get financial information from (e.g.) Yahoo Finance — other possibilities include Reuters or Bloomberg.

In [381]:
import random as rd

def get_financials(tk, max_sleep=2):
    """Get financial data from Yahoo Finance."""
    
    sleep(max_sleep*rd.random())    
    url=f'https://finance.yahoo.com/quote/{tk}/key-statistics?p={tk}'
    try:
        d = pd.concat(pd.read_html(url)).set_index(0).rename_axis(None).transpose()
        d['ticker'] = tk
        return d
    except Exception as e:
        pass

financials = list()
tickers = constituents[constituents.ticker.notna()].ticker.drop_duplicates()
for tk in tqdm_notebook(tickers, total=tickers.shape[0]):
    financials.append(get_financials(tk))
financials = pd.concat(financials).reset_index(drop=True)
financials.head()

,Market Cap (intraday) 5,Enterprise Value 3,Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6,Beta (5Y Monthly),...,"Short Ratio (May 14, 2012) 4","Short % of Float (May 14, 2012) 4","Short % of Shares Outstanding (May 14, 2012) 4","Shares Short (prior month Apr 12, 2012) 4","Shares Short (Nov 29, 2017) 4","Short Ratio (Nov 29, 2017) 4","Short % of Float (Nov 29, 2017) 4","Short % of Shares Outstanding (Nov 29, 2017) 4","Shares Short (prior month Oct 30, 2017) 4","Shares Short (prior month Feb 27, 2020) 4"
0,1.44B,1.38B,36.79,34.00,1.59,1.08,3.72,1.04,15.83,1.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,130.78M,249.68M,8.75,8.11,1.08,2.34,0.76,4.46,NaN,0.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,839.36M,1.17B,9.81,13.31,1.46,2.77,0.99,3.85,NaN,1.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,114.71M,82.32M,NaN,NaN,NaN,4.31,1.97,3.10,-4.32,1.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933.19M,902.56M,NaN,-785.00,-6.06,1.53,1.89,1.48,-206.30,1.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are a few things we need to do before working with this data. Most columns are string values, as a lot of numbers also include a unit such as M (for millions) or B (for billions). We should also clean up the names of the columns to make it easier to work with (e.g. the numbers at the end of some of the columns' name indicate footnotes on the Yahoo website and are of no use). Then we can merge it back in the original data.

In [419]:
import re

def clean_column_name(s):
    """Remove punctuation, convert to snake case, and remove footnote numbers."""
    
    table = str.maketrans(dict.fromkeys(string.punctuation)) 
    new_s = s.translate(table) 
    new_s = '_'.join([x.lower() for x in new_s.split()])
    return re.sub('_\d$','', new_s)

def str_to_num(s):
    """Convert data returned by yahoo finance to float."""
    
    if type(s)==str:
        nb, unit = s[:-1], s[-1]
        if unit=='M':
            return float(nb)*1e6
        if unit=='B':
            return float(nb)*1e9
        if unit in ['k', 'K']:
            return float(nb)*1e3
        if unit=='%':
            return float(nb.replace(',',''))/100
        if s=="∞":
            return np.inf
        
    return s
    
yf = financials.copy()
yf.columns = [clean_column_name(c) for c in yf.columns]
yf.loc[:,yf.columns!='ticker'] = yf.loc[:,yf.columns!='ticker'].applymap(str_to_num)
constituents = constituents.merge(yf, how='left', on='ticker')

## True PE ratio for the Russell 2000 index

Now we have everything we need to compute an estimation of the aggregate PE ratio of the Russell 2000 index, and we can also look at other metrics, such as the share of firms with negative earnings or price to book-value, EV/EBITDA, etc. We have successfully gathered information for about 93% of the index. As it turns about, 42% of Russell 2000 firms have negative earnings, and these firms represent about 27% of the free-float market cap of the index. Excluding these firms, we find a PE ratio of 18.5 — but when we include them, the aggregate PE ratio jumps to 50: US small caps stocks in the Russell 2000 index can be considered very expensive compared to how much money these firms actually make currently.

In [438]:
constituents['negative_income'] = constituents.net_income_avi_to_common_ttm.astype(float)<0

# Sample of firms for which we have all the necessary information
cols = ['market_cap_intraday', 'net_income_avi_to_common_ttm', 'negative_income']
sample = constituents[constituents.loc[:,cols].notna().all(axis=1)]
sample.loc[:, cols] = sample.loc[:,cols].astype(float)

# Unweighted stats
stats = sample.loc[:,cols].mean(axis=0).to_frame('unweighted')
# Weighted by index weights
stats['weighted'] = np.average(sample.loc[:,cols], axis=0,
                               weights=sample.loc[:,'weight'])
# Weighted, excluding firms with negative net income
stats['excl_neg'] = np.average(sample.loc[sample.negative_income.eq(0),cols], axis=0,
                               weights=sample.loc[sample.negative_income.eq(0),'weight'])
# PE ratio
stats.loc['pe',:] = stats.apply(lambda x: x.market_cap_intraday / x.net_income_avi_to_common_ttm)

print(f'Information available for {sample.weight.sum():.1f}% of the index')
display(stats)

Information available for 92.8% of the index


,unweighted,weighted,excl_neg
market_cap_intraday,1.128820e+09,2.171218e+09,2.241133e+09
net_income_avi_to_common_ttm,-6.177525e+06,4.347477e+07,1.211264e+08
negative_income,4.213452e-01,2.747419e-01,0.000000e+00
pe,-1.827301e+02,4.994202e+01,1.850244e+01
